<a href="https://colab.research.google.com/github/isaigordeev/sedt/blob/main/test_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel, BertPreTrainedModel, Trainer, TrainingArguments
from torch.utils.data import DataLoader

dataset_test = [] #данные от Алены
dataset_train = [] #данные от Алены

# Преобразование данных и создание DataLoader для тестового набора
dataset_test = np.array(dataset_test).reshape(-1, 3)  # Преобразование формы
dataset_test_tensor = torch.tensor(dataset_test, dtype=torch.float32)  # Преобразуем в тензор
dataloader_test = DataLoader(TensorDataset(dataset_test_tensor), batch_size=8, shuffle=True)
# Преобразование данных и создание DataLoader для тренировочного набора
dataset_train = np.array(dataset_train).reshape(-1, 3)  # Преобразование формы
dataset_train_tensor = torch.tensor(dataset_train, dtype=torch.float32)  # Преобразуем в тензор
dataloader_train = DataLoader(TensorDataset(dataset_train_tensor), batch_size=8, shuffle=True)


# Определение кастомной модели с несколькими линейными слоями
class BertForBinaryClassification(BertPreTrainedModel):
    def __init__(self, config, classifier_hidden_sizes=[256]):
        super(BertForBinaryClassification, self).__init__(config)

        self.bert = BertModel(config)

        # Создаем список слоев классификатора на основе переданных гиперпараметров
        classifier_layers = []
        input_size = config.hidden_size

        for hidden_size in classifier_hidden_sizes:
            classifier_layers.append(nn.Linear(input_size, hidden_size))
            classifier_layers.append(nn.ReLU())
            classifier_layers.append(nn.Dropout(config.hidden_dropout_prob))
            input_size = hidden_size

        classifier_layers.append(nn.Linear(input_size, 1))
        self.classifier = nn.Sequential(*classifier_layers)

        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        pooled_output = outputs.pooler_output
        logits = self.classifier(pooled_output)
        probs = torch.sigmoid(logits)

        loss = None
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            # Преобразуем метки в тип float и изменяем размерность
            labels = labels.float().view(-1, 1)
            loss = loss_fct(logits, labels)

        output = (probs,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output

# Инициализация модели
from transformers import BertConfig
config = BertConfig.from_pretrained('bert-base-uncased')

# Определение гиперпараметров классификатора
classifier_hidden_sizes = [512, 256, 128]

model = BertForBinaryClassification(
    config,
    classifier_hidden_sizes=classifier_hidden_sizes
)

# Заморозка всех слоев BERT, кроме последнего линейного слоя
for param in model.bert.parameters():
    param.requires_grad = False
for param in model.bert.encoder.layer[-1].parameters():
    param.requires_grad = True

# Настройка параметров обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='./logs',
)

# Инициализация тренера
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
)

trainer.train()
trainer.evaluate()


In [ ]:
import torch
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

dataset_test = [] #данные от Алены
dataset_train = [] #данные от Алены

# Преобразование данных и создание DataLoader для тестового набора
dataset_test = np.array(dataset_test).reshape(-1, 3)  # Преобразование формы
dataset_test_tensor = torch.tensor(dataset_test, dtype=torch.float32)  # Преобразуем в тензор
dataloader_test = DataLoader(TensorDataset(dataset_test_tensor), batch_size=8, shuffle=True)
# Преобразование данных и создание DataLoader для тренировочного набора
dataset_train = np.array(dataset_train).reshape(-1, 3)  # Преобразование формы
dataset_train_tensor = torch.tensor(dataset_train, dtype=torch.float32)  # Преобразуем в тензор
dataloader_train = DataLoader(TensorDataset(dataset_train_tensor), batch_size=8, shuffle=True)


# Инициализация модели
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2
)

# Заморозка всех слоев BERT, кроме последнего линейного слоя
for param in model.parameters():
    param.requires_grad = False
for param in model.encoder.layer[-1].parameters():
    param.requires_grad = True

# Настройка параметров обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='./logs',
    dataloader_shuffle=True,
)

# Инициализация тренера
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataloader_train,
    eval_dataset=dataloader_test,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()
